In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

# CSV 파일로부터 데이터 로드
fist_data = pd.read_csv('fist_data.csv')
open_data = pd.read_csv('open_data.csv')
wrong_data = pd.read_csv('wrong_data.csv')

# 데이터와 라벨 생성
X = pd.concat([fist_data, open_data, wrong_data], ignore_index=True)
y = ['fist'] * len(fist_data) + ['open'] * len(open_data) + ['wrong'] * len(wrong_data)

# 라벨 인코딩
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# 데이터를 3D 형태로 변환 (samples, timesteps, features)
X = np.array(X)
X = X.reshape((X.shape[0], 1, X.shape[1]))

# 데이터를 훈련셋과 테스트셋으로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# 훈련 데이터셋의 개수 출력
print(f"훈련 데이터셋 개수: {X_train.shape[0]}")

# 모델 정의
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    LSTM(50),
    Dropout(0.5),
    Dense(3, activation='softmax')  # 다중 분류를 위해 출력 뉴런을 3개로 설정하고 소프트맥스 활성화 함수 사용
])

# 옵티마이저 생성 (learning rate를 0.001로 설정)
optimizer = Adam(learning_rate=0.003)

# 모델 컴파일
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',  # 다중 분류를 위해 손실 함수를 sparse_categorical_crossentropy로 설정
              metrics=['accuracy'])

# 조기 종료 콜백
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# 모델 훈련 (에폭과 배치사이즈 설정 가능)
epochs = 30
batch_size = 22

history = model.fit(X_train, y_train, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    validation_split=0.2, 
                    callbacks=[early_stopping],
                    verbose=1)

# 모델 저장
model.save('fist_open_wrong_model.h5')

# 테스트 데이터로 모델 평가
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"테스트 정확도: {test_accuracy:.2f}")


훈련 데이터셋 개수: 6781
Epoch 1/30


C:\Users\kimha\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


247/247 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.7167 - loss: 0.6349 - val_accuracy: 0.9971 - val_loss: 0.0189
Epoch 2/30
247/247 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9949 - loss: 0.0227 - val_accuracy: 0.9993 - val_loss: 0.0016
Epoch 3/30
247/247 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9964 - loss: 0.0167 - val_accuracy: 1.0000 - val_loss: 7.9013e-04
Epoch 4/30
247/247 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9964 - loss: 0.0112 - val_accuracy: 1.0000 - val_loss: 4.2140e-04
Epoch 5/30
247/247 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9984 - loss: 0.0049 - val_accuracy: 1.0000 - val_loss: 0.0015
Epoch 6/30
247/247 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9987 - loss: 0.0051 - val_accuracy: 0.9993 - val_loss: 0.0012
Epoch 7/30
247/247 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9979 - loss: 0.0070 - val_accuracy: 1.0000 - val_loss: 2.8112e-04
Epoch 8/30
247/247 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9956 - loss: 0.0154 - val_accuracy: 1

53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9997 - loss: 2.6920e-04
테스트 정확도: 1.00


In [8]:
import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

# MediaPipe 초기화
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
mp_drawing_styles = mp.solutions.drawing_styles

# 모델 로드
model = tf.keras.models.load_model('fist_open_wrong_model.h5')

# 라벨 인코더 로드 및 적합
label_encoder = LabelEncoder()
label_encoder.fit(['fist', 'open', 'wrong'])

# 웹캠을 통해 영상을 캡처합니다.
cap = cv2.VideoCapture(0)

with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
  
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("카메라를 찾을 수 없습니다.")
      continue

    # 성능 향상을 위해 이미지 작성 가능성을 비활성화합니다.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)

    # 이미지 작성 가능성을 다시 활성화합니다.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.multi_hand_landmarks:
      predictions = []
      confidences = []

      for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())

        # 랜드마크 데이터 추출
        landmarks = [[lm.x, lm.y, lm.z] for lm in hand_landmarks.landmark]
        landmarks_flat = np.array(landmarks).flatten().tolist()

        # 예측을 위해 데이터 형태 변환
        landmarks_array = np.array([landmarks_flat])
        landmarks_array = landmarks_array.reshape((landmarks_array.shape[0], 1, landmarks_array.shape[1]))

        # 예측
        probabilities = model.predict(landmarks_array)[0]
        predicted_label_index = np.argmax(probabilities)
        predicted_label = label_encoder.inverse_transform([predicted_label_index])[0]
        confidence = probabilities[predicted_label_index]

        predictions.append(predicted_label)
        confidences.append(confidence)

      # 모든 예측이 동일한지 확인하고, 가장 높은 확률을 가진 예측을 선택
      if all(pred == predictions[0] for pred in predictions):
        final_prediction = predictions[np.argmax(confidences)]
      else:
        final_prediction = 'wrong'
      
      print(f"Prediction: {final_prediction}")

      # 이미지에 예측 결과 표시
      cv2.putText(image, f"Prediction: {final_prediction}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    else:
        final_prediction = "unknown"
        print(final_prediction)
        cv2.putText(image, f"Prediction: {final_prediction}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

    # 보기 편하게 이미지를 좌우 반전합니다.
    cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))
    
    # ESC 키를 누르면 종료합니다.
    if cv2.waitKey(5) & 0xFF == 27:
      break

cap.release()
cv2.destroyAllWindows()


unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown


C:\Users\kimha\anaconda3\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 322ms/step
Prediction: wrong
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Prediction: wrong
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Prediction: wrong
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Prediction: wrong
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Prediction: wrong
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Prediction: wrong
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Prediction: wrong
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Prediction: wrong
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Prediction: wrong
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Prediction: wrong
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Prediction: wrong
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Prediction: open
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Prediction: open
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Prediction: open
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Prediction: wrong
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Prediction: wrong
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Prediction: wrong
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Prediction: wrong
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Prediction: wrong
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Prediction: wrong
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Prediction: wrong
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Prediction: wrong
1/1 ━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Prediction: wrong
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Prediction: wrong
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Prediction: wrong
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Prediction: wrong
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Prediction: wrong
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Prediction: wrong
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
